## Imports

In [7]:
import duckdb
import os
import yaml

con = duckdb.connect()

In [8]:
with open("../config.yaml", "r") as yamlfile:
    keys = yaml.load(yamlfile, Loader=yaml.FullLoader)
    print("Read successful")

Read successful


## CAT

In [9]:
# con.execute("""SELECT * FROM 'category/*.parquet' limit 10;""").df()
con.execute("""INSTALL httpfs;""")
con.execute("""LOAD httpfs;""")

In [10]:
con.execute(f"""SET s3_access_key_id='{keys[0]['s3_access_key_id']}';""")
con.execute(f"""SET s3_secret_access_key='{keys[0]['s3_secret_access_key']}';""")
con.execute("""SET s3_region='us-east-1';""")


In [11]:
### Name of the bucket
root_bucket_path = "wfm-marketing-data-exchange"

### Use * to query ALL files in that folder path, otherwise point directly to the .csv file in the bucket sub folder.
path_to_file_or_folder = "E-Commerce Analytics/traffic-hits-test/"

test = con.execute(f"""SELECT * FROM read_csv('s3://{root_bucket_path}/{path_to_file_or_folder}', AUTO_DETECT=TRUE);""")

RuntimeError: Invalid Input Error: Error in file "s3://wfm-marketing-data-exchange/E-Commerce Analytics/traffic-hits-test/": CSV options could not be auto-detected. Consider setting parser options manually.

In [ ]:
test.df()

## Example Parquet Queries

In [ ]:
dma = con.execute("""SELECT * FROM 'dma/*.parquet' WHERE dma_name='San Francisco-Oakland-San Jose, CA';""").df()
dma

In [ ]:
con.execute("""SELECT COUNT(*) FROM dma;""").df()

## Example Parquet Queries WITH Temp Table Creation

In [ ]:
met = con.execute("""CREATE TEMP TABLE met AS SELECT * FROM 'metric/*.parquet' WHERE category='cross_shopping' and sub_category = 'weekly'""")

In [ ]:
con.execute("""SELECT COUNT(*) FROM met;""").df()

## Pre-join EDA

## Example Join df's

In [ ]:
join_df = con.execute("""SELECT * FROM plc 
                            --JOIN cat ON plc.category_ids=cat.category_id
                            JOIN dma ON plc.dma_id=dma.dma_code
                            JOIN met ON plc.spatial_id=met.spatial_id;""").df()

In [ ]:
# con.execute("""SELECT COUNT(*) FROM join_df;""")
g_df = con.execute("""SELECT metric, SUM(value) FROM join_df GROUP BY metric ORDER BY metric DESC;""").df()

In [ ]:
g_df.to_csv('g_df.csv')